<a href="https://colab.research.google.com/github/satya4038/Phonepe/blob/main/Phonepe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Phonepe Pulse Data Visualization and Exploration:
A User-Friendly Tool Using Streamlit and Plotly

In [ ]:
!git clone https://github.com/PhonePe/pulse.git

Cloning into 'pulse'...
remote: Enumerating objects: 9585, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9585 (delta 0), reused 1 (delta 0), pack-reused 9576
Receiving objects: 100% (9585/9585), 2.32 MiB | 6.22 MiB/s, done.
Resolving deltas: 100% (4219/4219), done.


In [ ]:
!apt-get install -y git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [ ]:
!pip install mysql-connector-python


In [ ]:
pip install GitPython


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade mysql-connector-python

In [ ]:
# Required libraries for the program
import pandas as pd
import json
import os
import mysql.connector
import git  # Make sure to install the 'GitPython' library (pip install GitPython)

# GitHub repository URL
github_repo_url = "https://github.com/username/repo.git"  # Replace with your repository URL
local_repo_path = "/path/to/local/repo"  # Replace with your desired local path

# Clone the GitHub repository
if not os.path.exists(local_repo_path):
    git.Repo.clone_from(github_repo_url, local_repo_path)

# Check the content of the directory
path = os.path.join(local_repo_path, "pulse/data/aggregated/transaction/country/india/state")

if os.path.exists(path):
    Agg_state_list = os.listdir(path)

    # This is to extract the data's to create a dataframe
    clm = {'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [], 'Transaction_count': [], 'Transaction_amount': []}
    for i in Agg_state_list:
        p_i = os.path.join(path, i + "/")
        Agg_yr = os.listdir(p_i)
        for j in Agg_yr:
            p_j = os.path.join(p_i, j + "/")
            Agg_yr_list = os.listdir(p_j)
            for k in Agg_yr_list:
                p_k = os.path.join(p_j, k)
                with open(p_k, 'r') as Data:
                    D = json.load(Data)
                    for z in D['data']['transactionData']:
                        Name = z['name']
                        count = z['paymentInstruments'][0]['count']
                        amount = z['paymentInstruments'][0]['amount']
                        clm['Transaction_type'].append(Name)
                        clm['Transaction_count'].append(count)
                        clm['Transaction_amount'].append(amount)
                        clm['State'].append(i)
                        clm['Year'].append(j)
                        clm['Quarter'].append(int(k.strip('.json')))

    # Successfully created a dataframe
    Agg_Trans = pd.DataFrame(clm)

    # Step 3: Database Insertion
    # Connect to MySQL database
    db_connection = mysql.connector.connect(
        host="localhost",
        user="shyam4038",
        password="shyam123",
        database="phonepe"
    )

    # Create a MySQL cursor
    cursor = db_connection.cursor()

    # Insert data into MySQL table (assuming a table named 'phonepe_transaction_data')
    create_table_query = """
    CREATE TABLE IF NOT EXISTS phonepe_transaction_data (
        State VARCHAR(255),
        Year VARCHAR(255),
        Quarter INT,
        Transaction_type VARCHAR(255),
        Transaction_count INT,
        Transaction_amount FLOAT
    )
    """
    cursor.execute(create_table_query)

    insert_query = f"INSERT INTO phonepe_transaction_data ({', '.join(Agg_Trans.columns)}) VALUES ({', '.join(['%s'] * len(Agg_Trans.columns))})"
    values = [tuple(row) for row in Agg_Trans.values]

    cursor.executemany(insert_query, values)

    # Commit changes and close connection
    db_connection.commit()
    cursor.close()
    db_connection.close()

    # Step 4: Dashboard Creation
    import streamlit as st
    import plotly.express as px

    # Load data from MySQL into Pandas DataFrame
    db_connection = mysql.connector.connect(
        host="localhost",
        user="shyam4038",
        password="shyam123",
        database="phonepe"
    )

    # Fetch data into a Pandas DataFrame (assuming the table name is 'phonepe_transaction_data')
    dashboard_data = pd.read_sql_query("SELECT * FROM phonepe_transaction_data", db_connection)

    # Step 5: Display Dashboard
    st.title("Phonepe Pulse Dashboard")

    # Create a Streamlit dropdown for user selection
    selected_metric = st.selectbox("Select a metric to display", dashboard_data.columns)

    # Create a Plotly figure based on user selection
    fig = px.scatter_geo(dashboard_data, locationmode="country names", locations="State", color=selected_metric, size=selected_metric, hover_name="State")
    st.plotly_chart(fig)
else:
    print(f"The specified path does not exist: {path}")
